In [83]:
import pandas as pd 
pd.set_option('display.max_colwidth', -1)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(12,8)})
%matplotlib inline
import glob
import csv
import os
import tarfile
import math
import numpy as np 
from tqdm import tqdm
import json
from progressive.bar import Bar
import pickle as pkl
import networkx as nx
from __future__ import unicode_literals

#Make sure to use python 2 kernerls since the pickles are incompatible with python 3 :( 

In [84]:
with open('../mappings/Bipartite_Drug_graph.pkl','rb') as f:
    drug_association_graph  = pkl.load(f)

with open('../mappings/Bipartite_Drug_category_graph.pkl','rb') as f:
    drug_cat_association_graph  = pkl.load(f)

In [85]:
print len(drug_association_graph.edges()) 
print len(drug_cat_association_graph.edges())
print len(drug_association_graph.nodes()) 
print len(drug_cat_association_graph.nodes())

7954
33493
4029
4047


In [86]:
#Old files till june 2019
files = glob.glob('../../BL_Work/openPrescribe/uncompressed/*.tar.gz')
print files

[u'../../BL_Work/openPrescribe/uncompressed/T201704PDPI BNFT.CSV.tar.gz', u'../../BL_Work/openPrescribe/uncompressed/T201403ADDR BNFT.CSV.tar.gz', u'../../BL_Work/openPrescribe/uncompressed/T201111CHEM SUBS.CSV.tar.gz', u'../../BL_Work/openPrescribe/uncompressed/T201704ADDR BNFT.CSV.tar.gz', u'../../BL_Work/openPrescribe/uncompressed/T201705ADDR BNFT.CSV.tar.gz', u'../../BL_Work/openPrescribe/uncompressed/T201210ADDR BNFT.CSV.tar.gz', u'../../BL_Work/openPrescribe/uncompressed/T201208CHEM SUBS.CSV.tar.gz', u'../../BL_Work/openPrescribe/uncompressed/T201807ADDR BNFT.CSV.tar.gz', u'../../BL_Work/openPrescribe/uncompressed/T201507PDPI BNFT.CSV.tar.gz', u'../../BL_Work/openPrescribe/uncompressed/T201605PDPI BNFT.CSV.tar.gz', u'../../BL_Work/openPrescribe/uncompressed/T201611ADDR BNFT.CSV.tar.gz', u'../../BL_Work/openPrescribe/uncompressed/T201110CHEM SUBS.CSV.tar.gz', u'../../BL_Work/openPrescribe/uncompressed/T201202PDPI BNFT.CSV.tar.gz', u'../../BL_Work/openPrescribe/uncompressed/T201312

In [87]:
files = glob.glob('../../BL_Work/openPrescribe/EPD*.zip')
print files

[u'../../BL_Work/openPrescribe/EPD_201912.zip', u'../../BL_Work/openPrescribe/EPD_202009.zip', u'../../BL_Work/openPrescribe/EPD_202003.zip', u'../../BL_Work/openPrescribe/EPD_202007.zip', u'../../BL_Work/openPrescribe/EPD_201910.zip', u'../../BL_Work/openPrescribe/EPD_202008.zip', u'../../BL_Work/openPrescribe/EPD_202006.zip', u'../../BL_Work/openPrescribe/EPD_202012.zip', u'../../BL_Work/openPrescribe/EPD_201909.zip', u'../../BL_Work/openPrescribe/EPD_202010.zip', u'../../BL_Work/openPrescribe/EPD_202004.zip', u'../../BL_Work/openPrescribe/EPD_201907.zip', u'../../BL_Work/openPrescribe/EPD_202001.zip', u'../../BL_Work/openPrescribe/EPD_202002.zip', u'../../BL_Work/openPrescribe/EPD_202101.zip', u'../../BL_Work/openPrescribe/EPD_201908.zip', u'../../BL_Work/openPrescribe/EPD_202005.zip', u'../../BL_Work/openPrescribe/EPD_202011.zip', u'../../BL_Work/openPrescribe/EPD_201911.zip', u'../../BL_Work/openPrescribe/EPD_202102.zip']


In [127]:
#Different functions to extract different data form the prescription 

def cleanStringofUTF(string):
    cleaned = string.replace('\xe8','e').replace('\xf6','o')
    return cleaned

def enrichdrugs(chem_dict , drugs):
    diabetes_drug_words = [drugs[k]['name'].lower() for k in drugs]
    for drug in chem_dict:
        Name = chem_dict[drug]['name'].replace('(','').replace(')','')
        slot1 = Name.lower().split('/')
        slot2 = Name.lower().split(' ')
        slot3 = Name.lower().split(' & ')
        common1 = set(diabetes_drug_words).intersection(slot1)
        common2 = set(diabetes_drug_words).intersection(slot2)
        common3 = set(diabetes_drug_words).intersection(slot3)
        
        if len(common1) > 0 or len(common2) > 0 or len(common3) > 0:
            print common1 , common2 , common3
            drugs[chem_dict[drug]['code']] = {'disease':'' , 'disease_given_drug':0.0 , 'matched_disease':'', 'name':chem_dict[drug]['name'].strip() }

            
            
def makeChemDict(BNF_Chem):
    chem_dict = {}
    for index, row in BNF_Chem.iterrows():
        chem_dict[row['UNII_drugbank']] = {}
        chem_dict[row['UNII_drugbank']]['name'] = row['NAME']
        chem_dict[row['UNII_drugbank']]['code'] = row['BNF_code']
    return chem_dict
    
def getDrugCategory(categorylist, BNF_Chem, drugbankDict):
    allMatched = []
    drugs = {}
    chem_dict = makeChemDict(BNF_Chem)
    
    for k in drugbankDict:
        if len(drugbankDict[k]['Categories']) > 0:
            for cat in drugbankDict[k]['Categories']:
                matched_memo = []
                catString = cat.values()[0]#.split('\u2014')[-1]
                t = catString.lower().strip()
                for categoryString in categorylist:
                    categoryString = categoryString.lower()
                    if t.find(categoryString) >= 0:
                        matched_memo.append(categoryString)
                if k in chem_dict:
                    if len(matched_memo) > 0:# == len(categorylist):
                        allMatched.append(k)
                        print chem_dict[k]
                        drugs[chem_dict[k]['code']] = {}
                        drugs[chem_dict[k]['code']]['name'] = chem_dict[k]['name']
                        drugs[chem_dict[k]['code']]['matched_cat'] = categorylist
    enrichdrugs(chem_dict,drugs)               
    return list(set(allMatched)) , drugs


def getDrugforDiseaseDrugbank(categorylist, BNF_Chem, drugbankDict):
    allMatched = []
    drugs = {}
    chem_dict = makeChemDict(BNF_Chem)
    
    for k in drugbankDict:
        if len(drugbankDict[k]['Associations']) > 0:
            for cat in drugbankDict[k]['Associations']:
                matched_memo = []
                catString = cat.values()[0]
                t = catString.lower().strip()
                for categoryString in categorylist:
                    categoryString = categoryString.lower()
                    if t.find(categoryString) >= 0:
                        matched_memo.append(categoryString)
                if k in chem_dict:
                    if len(matched_memo) > 0:
                        allMatched.append(k)
                        print chem_dict[k]
                        drugs[chem_dict[k]['code']] = {}
                        drugs[chem_dict[k]['code']]['name'] = chem_dict[k]['name']
                        drugs[chem_dict[k]['code']]['matched_cat'] = categorylist
    enrichdrugs(chem_dict,drugs)               
    return  allMatched , drugs


def findDrugsForDisease(Graph, Disease, BNF_Chem ):#,threshProb):
    chem_dict = makeChemDict(BNF_Chem)
    drugs = {}
    for e in Graph.edges(data=True):
        if (cleanStringofUTF(e[1]).lower().find(Disease.lower()) >=0) or (cleanStringofUTF(e[0]).lower().find(Disease.lower()) >= 0) :
            drugNode = ''
            matchedDisease = ''
            if Graph.node[e[0]]['type'] == 'symptom':
                drugNode = e[1]
                matchedDisease = e[0]
            else:
                drugNode = e[0]
                matchedDisease = e[1]
            drugs[Graph.node[drugNode]['Id']] = {}
            drugs[Graph.node[drugNode]['Id']]['name'] = drugNode
            drugs[Graph.node[drugNode]['Id']]['matched_disease'] = matchedDisease
            drugs[Graph.node[drugNode]['Id']]['disease'] = Disease
    enrichdrugs(chem_dict,drugs)
    return drugs


def generateConfidence(drugs,Graph):
    shared = []
    All = []
    denom = max(Graph.degree().values())
    for d in drugs:
        name = drugs[d]['name']
        for e in Graph.edges(data=True):
            if Graph.node[e[0]]['type'] == 'symptom':
                if e[1] == name:
                    shared.append(Graph.degree()[e[1]]-1)
                else:
                    continue
            else:
                
                if e[0] == name:
                    shared.append(Graph.degree()[e[0]]-1)
                else:
                    continue
#     shared = [float(k) for k in shared]
    num = [k for k in shared if k > 1]

    return float(len(num)+1.0)/float(len(shared)+1.0)
#     return float(len(num))/float(len(shared)) * 10.0
#     return len(num)
                
                
def findDrugsForCategory(Graph, Cat, BNF_Chem ):#,threshProb):
    chem_dict = makeChemDict(BNF_Chem)
    drugs = {}
    for e in Graph.edges(data=True):
        if (cleanStringofUTF(e[1]).lower().find(Cat.lower()) >=0) or (cleanStringofUTF(e[0]).lower().find(Cat.lower()) >= 0) :
            drugNode = ''
            matchedDisease = ''
            if Graph.node[e[0]]['type'] == 'category':
                drugNode = e[1]
                matchedDisease = e[0]
            else:
                drugNode = e[0]
                matchedDisease = e[1]
            print Graph.node[drugNode]['Id']
            drugs[Graph.node[drugNode]['Id']] = {}
            drugs[Graph.node[drugNode]['Id']]['name'] = drugNode
            drugs[Graph.node[drugNode]['Id']]['matched_cat'] = matchedDisease
            drugs[Graph.node[drugNode]['Id']]['category'] = Cat
    enrichdrugs(chem_dict,drugs)
    return drugs

def findDrugByName(Graph, name):
    foundDrugs = []
    for node in Graph.nodes(data=True):
        if node[1]['type'] == 'drug':
            if name.lower() in node[0].lower().split(' '):
                foundDrugs.append(node)
    return foundDrugs

def relabelNodes(graph):
    mapping = {}
    for node in graph.nodes():
        mapping[node] = cleanStringofUTF(node)
        
    return nx.relabel_nodes(graph,mapping)


def readChem(year,files):
    for f in files:
        if f.find(year)>0:
            if f.find('CHEM') > 0:
                df = pd.read_csv(f,compression='infer',header=None,skiprows=1)
                return df

def readAddress(year,files):
    for f in files:
        if f.find(year)>0:
            if f.find('ADDR') > 0:
                df = pd.read_csv(f,compression='infer',header=None,skiprows=1)
                return df

def readPrescriptions(year,files):
    for f in files:
        if f.find(year)>0:
            if f.find('PDPI') > 0:
                df = pd.read_csv(f,compression='infer',header=None,skiprows=1)
                return df
            
            
            
import inspect
import re

def extractPostCodesDict(addrDf):
    postcodeDict = {}
    for index,row in addrDf.iterrows():
        try:
            postcodeDict[row[1]] = row[7].strip()
        except:
            print "Found invalid entry"
    return postcodeDict

def checkIndex(index):
    if index%100 == 0:
        return True
    else:
        return False

def getPC(key , postcodeDict):
    codes = []
    for k in key:
        if k in postcodeDict:
            codes.append(postcodeDict[k])
        else:
            codes.append('')
    return pd.Series(codes,index=key.index)

def getPostcode(df,postcodeDict):
    df[10] = ''
    df[10] = df.groupby(2)[2].apply(getPC,postcodeDict)   
    return df

def getDrugFamily(key, diseaseMap):
    found = 'N/A'
    for dcode in diseaseMap: 
        if key.name.find(dcode) == 0:
            found = dcode
            break
    drug = [found]*len(key)
    return pd.Series(drug,index=key.index)

def getDisease(key, diseaseMap):
    found = 'N/A'
    for dcode in diseaseMap: 
        if key.name.find(dcode) == 0:
            found = diseaseMap[dcode]['disease'].replace('\"','').replace('+',' ')
            break
    drug = [found]*len(key)
    return pd.Series(drug,index=key.index)

def getDrug(key, diseaseMap):
    found = 'N/A'
    for dcode in diseaseMap: 
        if key.name.find(dcode) == 0:
            found = diseaseMap[dcode]['chemName']
            break
    drug = [found]*len(key)
    return pd.Series(drug,index=key.index)

def getDrugPotency(key):
    name = list(set(key))
    if len(name) > 1:
        print("found synonyms")
    text= name[-1]
    found = 0.0
    switch1 = text.find('mcg')
    switch2 = text.find('mg')
    switch3 = text.find('ml')
    
    if switch1 > 0 or switch2 > 0 or switch3 > 0:
        weight = re.findall(r'[0-9]*\.?[0-9]+', text)
        if len(weight) > 0:
            found = max([float(k) for k in weight])
            if switch1 > 0:
                found = found/1000.0
    potency = [found]*len(key)
    return pd.Series(potency,index=key.index)

def countSpecificDrugs(Df, drugs,GPs):
    df_slice = Df.groupby(3)[3].apply(getDrug,drugs)
    selected = df_slice[df_slice!='N/A']
    len(selected)
    df_selected =  Df.iloc[selected.index,:]
    df_selected = df_selected[df_selected[2].isin(GPs.keys())]
    return np.sum(df_selected[5])


def countSpecificDrugCosts(Df, drugs,GPs):
    df_slice = Df.groupby(3)[3].apply(getDrug,drugs)
    selected = df_slice[df_slice!='N/A']
    len(selected)
    df_selected =  Df.iloc[selected.index,:]
    df_selected = df_selected[df_selected[2].isin(GPs.keys())]
    return np.sum(df_selected[6])

def countDrugsByCategoryList(pdp,codes):
    total_drugs = 0.0
    drugs = None
    for name , group in pdp.groupby(3):
        for dcode in codes:
            if name.find(dcode) == 0:
                total_drugs+=np.sum(group[5])
    return total_drugs

def countPrescriptionsByCategoryList(pdp,codes):
    total_drugs = 0.0
    drugs = None
    for name , group in pdp.groupby(3):
        for dcode in codes:
            if name.find(dcode) == 0:
                total_drugs+=len(group)
    return total_drugs


def countDrugsCostByCategoryList(pdp,codes):
    total_drugs = 0.0
    for name , group in pdp.groupby(3):
        for dcode in codes:
            if name.find(dcode) == 0:
                total_drugs+=np.sum(group[7])
    return total_drugs

def countDrugsCostByGenerics(pdp,codes):
    total_drugs = 0.0
    generics= pdp[pdp[20] == 'AA']
    for name , group in generics.groupby(3):
        for dcode in codes:
            if name.find(dcode) == 0:
                total_drugs+=np.sum(group[7])
    return total_drugs

def compareCostsForGenericsAndBranded(pdp,codes):
    genericsCosts = {}
    brandedCosts = {}
    for name , group in pdp.groupby(16):
        for dcode in codes:
            if name == dcode:
                generics= group[group[20] == 'AA']
                if len(generics)>0:
                    total_drugs =np.sum(group[7])
                    generic_drugs = np.sum(generics[7])
                    brandedCosts[dcode] = total_drugs - generic_drugs
                    genericsCosts[dcode] = generic_drugs
                else:
                    print("Did not find any generic drugs")
                    continue
    return brandedCosts , genericsCosts

def func_Cost(potgroup):
    
    generics= potgroup.loc[potgroup[20] == 'AA']
    nonGenerics =  potgroup.loc[potgroup[20] != 'AA']

    minCost = np.min(generics[7])
    minpotdf = generics.loc[generics[7] == minCost]
    minQuant = np.min(minpotdf[8])
    if minQuant == 0:
        normalizer = minCost
    else:
        normalizer = float(minCost)/float(minQuant)
    
    if np.isnan(normalizer):
        normalizer = 1.0
    potgroup[21] = normalizer
    
    minCostBrand = np.min(nonGenerics[7])
    minpotdfBrand = nonGenerics.loc[nonGenerics[7] == minCostBrand]
    minQuantBrand = np.min(minpotdfBrand[8])
    
    if minQuantBrand == 0:
        unitNonGenericCost = minCostBrand
    else:
        unitNonGenericCost = float(minCostBrand)/float(minQuantBrand)
    
    if np.isnan(unitNonGenericCost):
        unitNonGenericCost = 1.0
    potgroup[22] = unitNonGenericCost
    if unitNonGenericCost > normalizer:
        potgroup[23] = float(unitNonGenericCost - normalizer)*potgroup[8]    
    return potgroup

def func_Drugs(group,codes):
    return group.groupby(15,as_index=False).apply(lambda df : func_Cost(df))
    

def computeSavingsNew(pdp,codes):
    pdp[21] = 0.0
    pdp[22] = 0.0
    pdp[23] = 0.0
    return pdp.groupby(16,as_index=False).apply(lambda df: func_Drugs(df , codes))


def computeSavings(pdp,codes):
    pdp[21] = 0.0
    pdp[22] = 0.0
    pdp[23] = 0.0
    for name , group in pdp.groupby(16):
        #we can remove this to allow computing savings across all drugs
        if name in codes:
            for pot , potgroup in group.groupby(15):
                generics= potgroup.loc[potgroup[20] == 'AA']
                nonGenerics =  potgroup.loc[potgroup[20] != 'AA']

                minCost = np.min(generics[7])
                minpotdf = generics.loc[generics[7] == minCost]
                minQuant = np.min(minpotdf[8])
                normalizer = float(minCost)/float(minQuant)
                potgroup[21] = normalizer

                minCostBrand = np.min(nonGenerics[7])
                minpotdfBrand = nonGenerics.loc[nonGenerics[7] == minCostBrand]
                minQuantBrand = np.min(minpotdfBrand[8])

                unitNonGenericCost = float(minCostBrand)/float(minQuantBrand)
                potgroup[22] = unitNonGenericCost
                
                if unitNonGenericCost > normalizer:
                    nonGenerics[23] = float(unitNonGenericCost - normalizer)*nonGenerics[8]


def countTotalDrugDosage(pdp,codes):
    total_drugs = 0.0
    for name , group in pdp.groupby(3):
        for dcode in codes:
            if name.find(dcode) == 0:
                total_drugs+=np.sum(group[19])
    return total_drugs

def normalizePills(keys):
    minima = np.min(keys)
    if minima > 0:
        potency = keys/minima
    else:
        potency = 1.0
    return pd.Series(potency,index=keys.index)
        
    
def normalizePotency(keys):
    minima = np.min(keys)
    if minima > 0:
        potency = keys/minima
    else:
        potency = 1.0
    return pd.Series(potency,index=keys.index)
    
def normalize(dataFrame):
    dataFrame[16] = dataFrame[3].str[:9]
    dataFrame[17] = dataFrame.groupby(16)[8].apply(normalizePills)
    dataFrame[18] = dataFrame.groupby(16)[15].apply(normalizePotency) 
    dataFrame[19] = dataFrame[18]*dataFrame[17]
    dataFrame[20] = dataFrame[3].str[9:11]
    dataFrame[21] = dataFrame[15]/dataFrame[8]
    
    
def normalize_new(dataFrame):
    dataFrame[16] = dataFrame['BNF_CODE'].str[:9]
    dataFrame[17] = dataFrame.groupby(16)['TOTAL_QUANTITY'].apply(normalizePills)
    dataFrame[18] = dataFrame.groupby(16)[15].apply(normalizePotency) 
    dataFrame[19] = dataFrame[18]*dataFrame[17]
    dataFrame[20] = dataFrame['BNF_CODE'].str[9:11]
    dataFrame[21] = dataFrame[15]/dataFrame['TOTAL_QUANTITY']
    
    
    
def doImportantMappings(Df, diseaseMap , symptomMap):
    #BNF family
    Df[11] = ''  
    Df[11] = Df.groupby(3)[3].apply(getDrugFamily,symptomMap)
    #Disease
    Df[12] = ''  
    Df[12] = Df.groupby(3)[3].apply(getDisease,diseaseMap)
    #Symptom
    Df[13] = ''  
    Df[13] = Df.groupby(3)[3].apply(getDisease,symptomMap)
    #Checm Name
    Df[14] = ''  
    Df[14] = Df.groupby(3)[3].apply(getDrug,symptomMap)
    #Chem potency
    Df[15] = ''  
    Df[15] = Df.groupby(3)[4].apply(getDrugPotency)

    normalize(Df)

    return Df



def doImportantMappings_new(Df, diseaseMap , symptomMap):
    #BNF family
    Df[11] = ''  
    Df[11] = Df.groupby('BNF_CODE')['BNF_CODE'].apply(getDrugFamily,symptomMap)
    #Disease
    Df[12] = ''  
    Df[12] = Df.groupby('BNF_CODE')['BNF_CODE'].apply(getDisease,diseaseMap)
    #Symptom
    Df[13] = ''  
    Df[13] = Df.groupby('BNF_CODE')['BNF_CODE'].apply(getDisease,symptomMap)
    #Checm Name
    Df[14] = ''  
    Df[14] = Df.groupby('BNF_CODE')['BNF_CODE'].apply(getDrug,symptomMap)
    #Chem potency
    Df[15] = ''  
    Df[15] = Df.groupby('BNF_CODE')['BNF_DESCRIPTION'].apply(getDrugPotency)

    normalize_new(Df)

    return Df



In [128]:
chem = pd.read_csv('../mappings/CHEM_MASTER_MAP.csv')
len(chem)

# chem = chem.dropna()

matched = chem[chem['UNII_drugbank']!='[]']

matchedMap = {}
for index,row in matched.iterrows():
    if row['UNII_drugbank'] not in matchedMap:
         matchedMap[row['UNII_drugbank']] = []
    matchedMap[row['UNII_drugbank']].append(row['BNF_code'])

In [129]:
matchedMap

{'JCV8365FWN': ['0206040AR'],
 '8KQ660G60G': ['0602020D0'],
 'O9U0F09D5X': ['0402010G0'],
 '0CD5FD6S2M': ['1304000N0'],
 '27Y3KJK423': ['0301030B0', '0301030C0', '0301030D0'],
 'LW0TIW155Z': ['1001010X0'],
 'Z633861EIM': ['0908010AM'],
 'MZ1IW7Q79D': ['0503021A0'],
 '5D06587D6R': ['0406000AB'],
 'J4ZHN3HBTE': ['0206020N0'],
 'S915P5499N': ['0901070B0'],
 'E89I1637KE': ['0601023I0'],
 'Q94YYU22B8': ['0405020E0'],
 '9100L32L2N': ['0601022B0'],
 '3G6A5W338E': ['0309020F0',
  '0404000D0',
  '0404000E0',
  '0407010AA',
  '0407010U0'],
 'CUB7JI0JV3': ['1203030D0', '1203040C0'],
 '742SXX0ICT': ['0205010Z0', '0704050R0'],
 'N0A3Z5XTC6': ['1105000S0'],
 '1XU104G55N': ['0212000AD'],
 '42H8PQ0NMJ': ['0206040AE'],
 '62XCK0G93T': ['0501110N0'],
 '61JJC8N5ZK': ['0504050T0'],
 'WV5EC51866': ['0405020S0'],
 '5Y5F15120W': ['0605010R0'],
 'GWH6IJ239E': ['0401030C0', '0401030E0'],
 'MDY902UXSR': ['020400070'],
 '953A26OA1Y': ['0307000K0'],
 'GNN1DV99GX': ['1001030F0'],
 'KXO2KT9N0G': ['0212000X0'],
 'L6O

In [130]:
drugbank_dict = json.load(open('../mappings/Drugbank_drugs_data.json','rb'))

In [131]:
condition_Association_map = {}
for k in drugbank_dict:
    if len(drugbank_dict[k]['Associations']) > 0:
        for ass in drugbank_dict[k]['Associations']:
            
            disease = ass[ass.keys()[0]]
            if disease not in condition_Association_map:
                condition_Association_map[disease] = []
            condition_Association_map[disease].append(drugbank_dict[k]['Name'])
    

category_Association_map = {}
for k in drugbank_dict:
    if len(drugbank_dict[k]['Categories']) > 0:
        for ass in drugbank_dict[k]['Categories']:
            
            disease = ass[ass.keys()[0]]
            if disease not in category_Association_map:
                category_Association_map[disease] = []
            category_Association_map[disease].append(drugbank_dict[k]['Name'])
            
commonKeys = set(drugbank_dict.keys()).intersection(matchedMap.keys())

category_BNF_map = {}
for k in commonKeys:
    if len(drugbank_dict[k]['Categories']) > 0:
        for ass in drugbank_dict[k]['Categories']:
            
            disease = ass[ass.keys()[0]]
            if disease not in category_BNF_map:
                category_BNF_map[disease] = []
            BNF_code = matched
            category_BNF_map[disease]+=matchedMap[k]

In [132]:
category_BNF_map

{u'Curare Alkaloids': ['1501050C0', '1501050S0'],
 u'Breast Cancer Resistance Protein Inhibitors': ['0802040AI',
  '0503032H0',
  '0503010G0',
  '0503010U0',
  '0503032G0',
  '0502000M0',
  '0502010C0',
  '0503010AI',
  '0503010AK',
  '0503010AL',
  '0501070AD',
  '0901040C0',
  '0503032E0'],
 u'Prostacyclin Analogues': ['0205010V0'],
 u'UGT1A1 Substrates': ['0801040D0',
  '0602010V0',
  '0503010E0',
  '0503010AE',
  '1501010L0',
  '0407020B0',
  '0410030A0',
  '1002020H0',
  '0801050BJ',
  '0212000L0',
  '0212000M0',
  '0212000P0',
  '020400080',
  '1001010P0',
  '100101070',
  '0408010H0',
  '0204000I0',
  '1501070G0',
  '1902070Q0',
  '0901030K0',
  '040702040',
  '0604011D0',
  '0702010E0',
  '0703010E0',
  '0703010F0',
  '0703010G0',
  '0703010H0',
  '0703010I0',
  '0703010P0',
  '0703010Q0',
  '0408010AD',
  '0212000B0',
  '1001010L0',
  '100302010',
  '0212000C0',
  '1001010I0',
  '1104020G0',
  '0410030E0',
  '0402030Q0',
  '040801020',
  '0601021P0',
  '0803041AA',
  '0503010A

In [133]:
chem[chem['BNF_code'].isin(category_BNF_map['Caffeine'])]

,Unnamed: 0,BNF_code,NAME,UNII_drugbank
712,712,0309020F0,Caffeine & Sodium Iodide,3G6A5W338E
894,894,0404000D0,Caffeine,3G6A5W338E
895,895,0404000E0,Caffeine Citrate,3G6A5W338E
948,948,0407010AA,Aspirin & Caffeine,3G6A5W338E
966,966,0407010U0,Paracetamol & Caffeine,3G6A5W338E


In [134]:
drug_dep = findDrugsForDisease(drug_association_graph, 'depression',chem)

set([]) set([u'amoxapine']) set([])
set([]) set([u'aripiprazole']) set([])
set([]) set([u'perphenazine']) set([])
set([]) set([u'lamotrigine']) set([])
set([]) set([u'isocarboxazid']) set([])
set([]) set([u'oxazepam']) set([])
set([]) set([u'pindolol']) set([])
set([]) set([u'mirtazapine']) set([])
set([]) set([u'chlordiazepoxide']) set([])


In [135]:
dep_shared = generateConfidence(drug_dep,drug_association_graph)

In [136]:
drug_dep

{'020400010': {u'disease': u'',
  u'disease_given_drug': 0.0,
  u'matched_disease': u'',
  u'name': 'Pindolol With Diuretic'},
 '0204000P0': {u'disease': u'depression',
  u'matched_disease': u'Depression',
  u'name': u'Pindolol'},
 '0305010W0': {u'disease': u'depression',
  u'matched_disease': u'Postoperative respiratory depression',
  u'name': u'Doxapram Hydrochloride'},
 '0401020B0': {u'disease': u'depression',
  u'matched_disease': u'Depression',
  u'name': u'Buspirone Hydrochloride'},
 '0401020D0': {u'disease': u'depression',
  u'matched_disease': u'Depression',
  u'name': u'Chlordiazepoxide'},
 '0401020E0': {u'disease': u'',
  u'disease_given_drug': 0.0,
  u'matched_disease': u'',
  u'name': 'Chlordiazepoxide Hydrochloride'},
 '0401020T0': {u'disease': u'',
  u'disease_given_drug': 0.0,
  u'matched_disease': u'',
  u'name': 'Oxazepam'},
 '0402010AD': {u'disease': u'depression',
  u'matched_disease': u'Psychotic Depression',
  u'name': u'Aripiprazole'},
 '0402010Q0': {u'disease': u

In [137]:
antiDep_BNF =  chem[chem['BNF_code'].isin(drug_dep)]

In [138]:
antiDep_BNF

,Unnamed: 0,BNF_code,NAME,UNII_drugbank
285,285,0204000P0,Pindolol,BJ4HF6IU1D
295,295,020400010,Pindolol With Diuretic,BJ4HF6IU1D
673,673,0305010W0,Doxapram Hydrochloride,94F3830Q73
765,765,0401020B0,Buspirone Hydrochloride,TK65WKS8HL
767,767,0401020D0,Chlordiazepoxide,6RZ6XEZ3CR
768,768,0401020E0,Chlordiazepoxide Hydrochloride,6RZ6XEZ3CR
776,776,0401020T0,Oxazepam,6GOW6DWN2A
791,791,0402010AD,Aripiprazole,82VFR53I78
812,812,0402010Q0,Perphenazine,FTA7XXY4EZ
831,831,0402020AD,Aripiprazole,82VFR53I78


In [139]:
drug_association_graph = relabelNodes(drug_association_graph)

In [155]:
diseases = [
 "anxiety",
 "rheumatoid",
 "depression",
 "diabetes"]

In [156]:
DiseaseDrugs = {}
for d in diseases:
    drugs = findDrugsForDisease(drug_association_graph,d ,chem)
#     _ , drugs = getDrugforDiseaseDrugbank([d] ,chem,drugbank_dict)
    for drug in drugs:
        DiseaseDrugs[drug] = {}
        DiseaseDrugs[drug]['chemName'] = drugs[drug]['name']
        DiseaseDrugs[drug]['disease'] = d

set([]) set([u'gabapentin']) set([])
set([]) set([u'lorazepam']) set([])
set([]) set([u'perphenazine']) set([])
set([]) set([u'diazepam']) set([])
set([]) set([u'oxazepam']) set([])
set([]) set([u'alprazolam']) set([])
set([]) set([u'quetiapine']) set([])
set([]) set([u'bromazepam']) set([])
set([]) set([u'mirtazapine']) set([])
set([]) set([u'trifluoperazine']) set([])
set([]) set([u'venlafaxine']) set([])
set([]) set([u'chlordiazepoxide']) set([])
set([]) set([u'escitalopram']) set([])
set([]) set([u'meprobamate']) set([])
set([]) set([u'nabumetone']) set([])
set([]) set([u'penicillamine']) set([])
set([]) set([u'celecoxib']) set([])
set([]) set([u'leflunomide']) set([])
set([]) set([u'tetracosactide']) set([])
set([]) set([u'abatacept']) set([])
set([]) set([u'dexamethasone']) set([])
set([]) set([u'aspirin']) set([])
set([]) set([u'tolmetin']) set([])
set([]) set([u'tenoxicam']) set([])
set([]) set([u'rituximab']) set([])
set([]) set([u'aspirin']) set([])
set([]) set([u'naproxen'])

In [159]:
SymptomDrugs = json.load(open('../mappings/SymptomsMatched.json','rb'))
symptoms = [k.replace('+',' ').replace('"','').lower() for k in SymptomDrugs.keys()]
print symptoms

[u'impairment level of vision', u'backache', u'dyssomnia', u'loss of hair', u'mood swings', u'tired', u'weight gain', u'emotional upset', u'disorientated', u'clouded consciousness', u'stomach ache', u'increased thirst', u'sleep deprivation', u'heartburn', u'abdominal pain', u'swelling', u'feeling agitated', u'impairment of balance', u'paresthesia', u'blurring of visual image', u'feeling angry', u'loss of appetite', u'pain', u'tremor', u'rectal hemorrhage', u'abnormal gait', u'impotence', u'eruption of skin', u'feeling hopeless', u'indigestion', u"parkinson's disease", u'trigeminal neuralgia', u'bowel dysfunction', u'parkinsonism', u'delusional disorder', u'muscle pain', u'restlessness', u'malaise', u'thoughts of self harm', u'dementia', u'depressive disorder', u'hypothyroidism', u'reduced mobility', u'jaundice', u'lack of energy', u'joint pain', u'vertigo', u'headache', u'tinnitus', u'decrease in appetite', u'insomnia', u'multiple sclerosis', u'dry skin', u'blood in urine', u'increased

In [160]:
symptoms_matched = {}
for d in symptoms:
    drugs = findDrugsForDisease(drug_association_graph,d, chem)
    for drug in drugs:
        symptoms_matched[drug] = {}
        symptoms_matched[drug]['chemName'] = drugs[drug]['name']
        symptoms_matched[drug]['disease'] = d

set([]) set([u'capsaicin']) set([])
set([]) set([u'tenoxicam']) set([])
set([]) set([u'naproxen']) set([])
set([]) set([u'camphor']) set([])
set([]) set([u'caffeine']) set([])
set([]) set([u'modafinil']) set([])
set([]) set([u'orlistat']) set([])
set([]) set([u'topiramate']) set([])
set([]) set([u'magaldrate']) set([])
set([]) set([u'omeprazole']) set([])
set([]) set([u'cimetidine']) set([])
set([]) set([u'famotidine']) set([])
set([]) set([u'esomeprazole']) set([])
set([]) set([u'paracetamol', u'caffeine']) set([u'paracetamol'])
set([]) set([u'nortriptyline']) set([])
set([]) set([u'capsaicin']) set([])
set([]) set([u'denosumab']) set([])
set([]) set([u'gabapentin']) set([])
set([]) set([u'celecoxib']) set([])
set([]) set([u'baclofen']) set([])
set([]) set([u'dextropropoxyphene']) set([])
set([]) set([u'buprenorphine']) set([])
set([]) set([u'bromfenac']) set([])
set([]) set([u'oxandrolone']) set([])
set([]) set([u'aspirin']) set([])
set([]) set([u'tolmetin']) set([])
set([]) set([u't

set([]) set([u'adrenaline']) set([])
set([]) set([u'droperidol']) set([])
set([]) set([u'fructose']) set([])
set([]) set([u'lorazepam']) set([])
set([]) set([u'aprepitant']) set([])
set([]) set([u'granisetron']) set([])
set([]) set([u'perphenazine']) set([])
set([]) set([u'nabilone']) set([])
set([u'carbidopa']) set([]) set([])
set([]) set([u'haloperidol']) set([])
set([]) set([u'mirtazapine']) set([])
set([]) set([u'clozapine']) set([])
set([]) set([u'paracetamol']) set([u'paracetamol'])
set([]) set([u'tetracosactide']) set([])
set([]) set([u'aspirin']) set([])
set([u'ciprofloxacin']) set([]) set([])
set([]) set([u'aspirin']) set([])
set([]) set([u'naproxen']) set([])
set([]) set([u'sulfadiazine']) set([])
set([]) set([u'tetracycline']) set([])
set([]) set([u'erythromycin']) set([])
set([]) set([u'ibuprofen']) set([])
set([]) set([u'colchicine']) set([])
set([]) set([u'hydrochlorothiazide']) set([])
set([]) set([u'torasemide']) set([])
set([]) set([u'nifedipine']) set([])
set([]) set(

In [157]:
DiseaseDrugs

{'0103010S0': {u'chemName': u'Ranitidine Bismuth Citrate',
  u'disease': u'rheumatoid'},
 '0103010T0': {u'chemName': u'Ranitidine Hydrochloride',
  u'disease': u'rheumatoid'},
 '0105010E0': {u'chemName': 'Sulfasalazine', u'disease': u'rheumatoid'},
 '0202010L0': {u'chemName': 'Hydrochlorothiazide', u'disease': u'diabetes'},
 '0202080M0': {u'chemName': u'Hydrochlorothiazide/Potassium',
  u'disease': u'diabetes'},
 '020400010': {u'chemName': 'Pindolol With Diuretic',
  u'disease': u'depression'},
 '0204000P0': {u'chemName': u'Pindolol', u'disease': u'depression'},
 '0204000R0': {u'chemName': u'Propranolol Hydrochloride',
  u'disease': u'anxiety'},
 '0209000V0': {u'chemName': 'Dipyridamole & Aspirin',
  u'disease': u'rheumatoid'},
 '0212000AD': {u'chemName': u'Colesevelam Hydrochloride',
  u'disease': u'diabetes'},
 '0302000T0': {u'chemName': u'Triamcinolone Acetonide',
  u'disease': u'rheumatoid'},
 '0304010J0': {u'chemName': u'Hydroxyzine Hydrochloride',
  u'disease': u'anxiety'},
 '030

In [140]:
len([k for k in drug_association_graph.nodes(data=True) if k[1]['type'] == 'symptom'])

2662

In [141]:
monthListAll = []
for f in files: 
    weight = re.findall(r'[0-9]*\.?[0-9]+', f)
    monthListAll.append(weight[0])
monthListAll = list(set(monthListAll))
monthListAll.sort()
print monthListAll

[u'201907', u'201908', u'201909', u'201910', u'201911', u'201912', u'202001', u'202002', u'202003', u'202004', u'202005', u'202006', u'202007', u'202008', u'202009', u'202010', u'202011', u'202012', u'202101', u'202102']


In [142]:
GP_META = pd.read_csv('../mappings/epraccur.csv',header=None)
GP_META.rename(columns={0:'BP_code',
                        1:'Name',
                        2:'Grouping',
                        3:'National_geo',
                        4:'Addr1',
                        5:'Addr2',
                        6:'Addr3',
                        7: 'Addr4',
                        8:'Addr5',
                        9:'Postcode',
                        10:'Open',
                        11:'Closed',
                        12:'Status',
                        13:'Org type code',
                        14:'Commissioner',
                        15:'Join provider',
                        16:'Left provider',
                        17:'Contact',
                        18:'Null1',
                        19:'Null2',
                        20:'Null3',
                        21:'Amended',
                        22:'Null4',
                        23:'Provider',
                        24:'Null5',
                        25:'Setting',
                        26:'Null6'}, 
                 inplace=True)

In [143]:
Open_filtered_GPs = {}
for index,row in GP_META.iterrows():
    if row ['Setting'] == 4 and row['Status'] =='A':
        Open_filtered_GPs[row['BP_code']] = row['Postcode'].strip()

In [144]:
import logging
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='Serializing.log', mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)

In [145]:
pdp_old = pd.read_csv('../../BL_Work/openPrescribe/uncompressed/T201206PDPI BNFT.csv.tar.gz',compression='infer',header=None,skiprows=1)

In [146]:
pdp_old.head()

,0,1,2,3,4,5,6,7,8,9,10
0,Q30,5D7,A86003,0101010G0AAABAB,Co-Magaldrox_Susp 195mg/220mg/5ml S/F,18.0,52.24,48.71,10000.0,201206.0,
1,Q30,5D7,A86003,0101010N0AAAAAA,Antacid/Oxetacaine_Oral Susp S/F,1.0,97.42,89.77,300.0,201206.0,
2,Q30,5D7,A86003,0101010R0AAABAB,Simeticone_Susp 40mg/ml S/F,2.0,4.90,4.58,100.0,201206.0,
3,Q30,5D7,A86003,0101021B0AAAHAH,Gppe Liq_Gaviscon S/F,2.0,4.45,4.17,1000.0,201206.0,
4,Q30,5D7,A86003,0101021B0AAALAL,Sod Algin/Pot Bicarb_Susp (Aniseed) S/F,3.0,11.80,10.97,1300.0,201206.0,


In [147]:
pdp = pd.read_csv('../../BL_Work/openPrescribe/'+'EPD_'+str(monthListAll[0]) +'.zip')

In [148]:
pdp[1000:1009]

,YEAR_MONTH,REGIONAL_OFFICE_NAME,REGIONAL_OFFICE_CODE,AREA_TEAM_NAME,AREA_TEAM_CODE,PCO_NAME,PCO_CODE,PRACTICE_NAME,PRACTICE_CODE,ADDRESS_1,...,BNF_CODE,BNF_DESCRIPTION,BNF_CHAPTER_PLUS_CODE,QUANTITY,ITEMS,TOTAL_QUANTITY,ADQUSAGE,NIC,ACTUAL_COST,UNIDENTIFIED
1000,201907,LONDON,Y56,NORTH WEST LONDON AREA,Q62,CENTRAL LONDON (WESTMINSTER) CCG,09A00,LISSON GROVE HEALTH CENTRE,E87011,LISSON GROVE HEALTH CTR,...,0406000T0AAAKAK,Prochlorperazine 3mg buccal tablets,04: Central Nervous System,20.0,1,20.0,6.66667,12.92,12.12241,N
1001,201907,LONDON,Y56,NORTH WEST LONDON AREA,Q62,CENTRAL LONDON (WESTMINSTER) CCG,09A00,LISSON GROVE HEALTH CENTRE,E87011,LISSON GROVE HEALTH CTR,...,0407010H0AAAAAA,Paracetamol 500mg capsules,04: Central Nervous System,42.0,4,168.0,28.00000,4.64,4.76279,N
1002,201907,LONDON,Y56,NORTH WEST LONDON AREA,Q62,CENTRAL LONDON (WESTMINSTER) CCG,09A00,LISSON GROVE HEALTH CENTRE,E87011,LISSON GROVE HEALTH CTR,...,0407010H0AAAAAA,Paracetamol 500mg capsules,04: Central Nervous System,100.0,5,500.0,83.33333,13.75,12.84355,N
1003,201907,LONDON,Y56,NORTH WEST LONDON AREA,Q62,CENTRAL LONDON (WESTMINSTER) CCG,09A00,LISSON GROVE HEALTH CENTRE,E87011,LISSON GROVE HEALTH CTR,...,0407010H0AAAAAA,Paracetamol 500mg capsules,04: Central Nervous System,60.0,1,60.0,10.00000,1.65,1.64619,N
1004,201907,LONDON,Y56,NORTH WEST LONDON AREA,Q62,CENTRAL LONDON (WESTMINSTER) CCG,09A00,LISSON GROVE HEALTH CENTRE,E87011,LISSON GROVE HEALTH CTR,...,0407010H0AAAMAM,Paracetamol 500mg tablets,04: Central Nervous System,56.0,29,1624.0,270.66667,31.03,32.10408,N
1005,201907,LONDON,Y56,NORTH WEST LONDON AREA,Q62,CENTRAL LONDON (WESTMINSTER) CCG,09A00,LISSON GROVE HEALTH CENTRE,E87011,LISSON GROVE HEALTH CTR,...,0407010H0AAAMAM,Paracetamol 500mg tablets,04: Central Nervous System,32.0,12,384.0,64.00000,7.32,6.95323,N
1006,201907,LONDON,Y56,NORTH WEST LONDON AREA,Q62,CENTRAL LONDON (WESTMINSTER) CCG,09A00,LISSON GROVE HEALTH CENTRE,E87011,LISSON GROVE HEALTH CTR,...,0407010H0AAAMAM,Paracetamol 500mg tablets,04: Central Nervous System,100.0,91,9100.0,1516.66666,173.81,162.69653,N
1007,201907,LONDON,Y56,NORTH WEST LONDON AREA,Q62,CENTRAL LONDON (WESTMINSTER) CCG,09A00,LISSON GROVE HEALTH CENTRE,E87011,LISSON GROVE HEALTH CTR,...,0407010H0AAAMAM,Paracetamol 500mg tablets,04: Central Nervous System,224.0,12,2688.0,448.00000,51.34,47.87279,N
1008,201907,LONDON,Y56,NORTH WEST LONDON AREA,Q62,CENTRAL LONDON (WESTMINSTER) CCG,09A00,LISSON GROVE HEALTH CENTRE,E87011,LISSON GROVE HEALTH CTR,...,0407010H0AAAMAM,Paracetamol 500mg tablets,04: Central Nervous System,200.0,8,1600.0,266.66667,30.55,28.49749,N


In [149]:
print set(pdp['BNF_CHAPTER_PLUS_CODE'])

set(['03: Respiratory System', '07: Obstetrics, Gynaecology and Urinary-Tract Disorders', '13: Skin', '06: Endocrine System', '19: Other Drugs and Preparations', '12: Ear, Nose and Oropharynx', '08: Malignant Disease and Immunosuppression', '01: Gastro-Intestinal System', '05: Infections', '02: Cardiovascular System', '10: Musculoskeletal and Joint Diseases', '21: Appliances', '18: Preparations used in Diagnosis', '11: Eye', '09: Nutrition and Blood', '22: Incontinence Appliances', '14: Immunological Products and Vaccines', '04: Central Nervous System', '15: Anaesthesia', '20: Dressings', '23: Stoma Appliances'])


In [150]:
cat_filters = ['15: Anaesthesia', '20: Dressings', '23: Stoma Appliances', '21: Appliances','18: Preparations used in Diagnosis']

In [151]:
pdp = pdp.dropna()
pdp_filtered = pdp[pdp['PRACTICE_CODE'].isin(Open_filtered_GPs.keys())].copy(deep=True)
pdp_filtered = pdp[~pdp['BNF_CHAPTER_PLUS_CODE'].isin(cat_filters)]

In [152]:
len(pdp) , len(pdp_filtered)

(14127546, 12853369)

In [161]:
pdp_filtered = doImportantMappings_new(pdp_filtered,DiseaseDrugs,symptoms_matched)

/10TBdrive/sagarj/anaconda3/envs/env_py2/lib/python2.7/site-packages/ipykernel_launcher.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/10TBdrive/sagarj/anaconda3/envs/env_py2/lib/python2.7/site-packages/ipykernel_launcher.py:481: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/10TBdrive/sagarj/anaconda3/envs/env_py2/lib/python2.7/site-packages/ipykernel_launcher.py:483: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [165]:
print(pdp_filtered.iloc[51000])

YEAR_MONTH                      201907                                   
REGIONAL_OFFICE_NAME            LONDON                                   
REGIONAL_OFFICE_CODE            Y56                                      
AREA_TEAM_NAME                  NORTH EAST LONDON AREA                   
AREA_TEAM_CODE                  Q61                                      
PCO_NAME                        ENFIELD CCG                              
PCO_CODE                        07X00                                    
PRACTICE_NAME                   KEATS SURGERY                            
PRACTICE_CODE                   F85010                                   
ADDRESS_1                       KEATS SURGERY                            
ADDRESS_2                       290A CHURCH STREET                       
ADDRESS_3                       EDMONTON                                 
ADDRESS_4                       LONDON                                   
POSTCODE                        N9 9HJ

In [167]:
print pdp_filtered[pdp_filtered[16]=='0403040X0'].iloc[0]

YEAR_MONTH                      201907                          
REGIONAL_OFFICE_NAME            LONDON                          
REGIONAL_OFFICE_CODE            Y56                             
AREA_TEAM_NAME                  NORTH WEST LONDON AREA          
AREA_TEAM_CODE                  Q62                             
PCO_NAME                        CENTRAL LONDON (WESTMINSTER) CCG
PCO_CODE                        09A00                           
PRACTICE_NAME                   MAIDA VALE MEDICAL CENTRE       
PRACTICE_CODE                   E87010                          
ADDRESS_1                       MAIDA VALE HEALTH CENTRE        
ADDRESS_2                       40 BIDDULPH MANSIONS            
ADDRESS_3                       210 ELGIN AVENUE                
ADDRESS_4                       LONDON                          
POSTCODE                        W9 1HT                          
BNF_CHEMICAL_SUBSTANCE          0403040X0                       
CHEMICAL_SUBSTANCE_BNF_DE

In [168]:
# USE this for pre 2019-06 data
# monthList = monthListAll
# # monthList = [u'201601', u'201602', u'201603',]
# for month in tqdm(monthList):
#     filename = '../BL_Work/openPrescribe/serialized/' + month + '.gz'
#     if not os.path.exists(filename):
#         logging.info("working with File: "+str(filename))
#         pdp = readPrescriptions(month,files)
#         pdp = pdp.dropna()
#         pdp = getPostcode(pdp , postcodes)
#         pdp_filtered = pdp[pdp[2].isin(Open_filtered_GPs.keys())].copy(deep=True)
#         pdp_filtered = doImportantMappings(pdp_filtered,DiseaseDrugs,symptoms_matched)
#         pdp_filtered.to_csv(filename,compression='gzip')

In [171]:
# USE this for post 2019-06 data
monthList = monthListAll
# monthList = [u'201601', u'201602', u'201603',]
for month in tqdm(monthList[1:]):
    filename = '../../BL_Work/openPrescribe/serialized/' + month + '.gz'
    if not os.path.exists(filename):
        logging.info("working with File: "+str(filename))
        pdp = pd.read_csv('../../BL_Work/openPrescribe/'+'EPD_'+str(month) +'.zip')
        pdp = pdp.dropna()
        pdp_filtered = pdp[pdp['PRACTICE_CODE'].isin(Open_filtered_GPs.keys())].copy(deep=True)
        pdp_filtered = pdp[~pdp['BNF_CHAPTER_PLUS_CODE'].isin(cat_filters)]
        pdp_filtered = doImportantMappings_new(pdp_filtered,DiseaseDrugs,symptoms_matched)
        pdp_filtered.to_csv(filename,compression='gzip')
        del pdp_filtered
        del pdp



  0%|          | 0/19 [00:00<?, ?it/s]/home/sagar/anaconda3/envs/env_py2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/10TBdrive/sagarj/anaconda3/envs/env_py2/lib/python2.7/site-packages/ipykernel_launcher.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/10TBdrive/sagarj/anaconda3/envs/env_py2/lib/python2.7/site-packages/ipykernel_launcher.py:481: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable